## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
from math import ceil
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [2]:
effect_size = sms.proportion_effectsize(0.20, 0.19)

In [3]:
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )                                               

required_n = ceil(required_n)                         

print(f'Нам буде потрібно мінімум {required_n} спостережень для кожної групи.')

Нам буде потрібно мінімум 24638 спостережень для кожної групи.


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [4]:
data_path = 'C:\\Users\\Svitlana\\data\\Modul 14\\cookie_cats.csv'
data = pd.read_csv(data_path)

In [5]:
data.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [6]:
avg_r7=data.groupby('version')['retention_7'].mean().reset_index()

In [7]:
print(avg_r7)

   version  retention_7
0  gate_30     0.190201
1  gate_40     0.182000


Версія з утриманням воріт на 30 рівні показує краще утримання через 7 днів.
Нульова гіпотеза буде що утримання через 7 днів не відрізняється між gate_30  та gate_40
Альтернативна гіпотеза що утримання через 7 днів у версії gate_30 вища ніж у версії gate_40. 
Тест у нас правосторонній.

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [9]:
table=pd.crosstab(data['version'], data['retention_7'])

In [10]:
table

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [11]:
session_counts = data['userid'].value_counts(ascending=False) 
multi_users = session_counts[session_counts > 1].count() 
print(f'Є {multi_users} користувачів, які зустрічаються кілька разів у наборі даних.')

Є 0 користувачів, які зустрічаються кілька разів у наборі даних.


In [12]:
control_results = data[data['version'] == 'gate_30']['retention_7']
treatment_results = data[data['version'] == 'gate_40']['retention_7']

n_con = control_results.count()
n_treat = treatment_results.count()

successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
confint = proportion_confint(successes, nobs=nobs, alpha=0.05)
confint_con = proportion_confint(successes[0], nobs[0], alpha=0.05, method='normal')
confint_treat = proportion_confint(successes[1], nobs[1], alpha=0.05, method='normal')

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи control: [{confint_con[0]:.3f}, {confint_con[1]:.3f}]')
print(f'Довірчий інтервал 95% для групи treatment: [{confint_treat[0]:.3f}, {confint_treat[1]:.3f}]')




z statistic: 3.16
p-value: 0.002
Довірчий інтервал 95% для групи control: [0.187, 0.194]
Довірчий інтервал 95% для групи treatment: [0.178, 0.186]


Висновок: Оскільки Оскільки p-value 0.002 < α 0.05, ми відхиляємо H₀. Це означає, що утримання на 7ий день у групах статистично значуще відрізняється. У групі gate_30 має вище утримання ніж у gate_40. Довірчі інтервали майже торкаються, але не перекриваються, що підтверджує попередній висновок.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


Тест Хі-квадрат квадрат може визначити чи є вплив чи немає. Отже нульова гіпотеза буде що версія гри  не вливає на утримання 7ий день, альтернативна гіпотеза: залежність існує. 

In [13]:
chi2, p, dof, expected = stats.chi2_contingency(table.values)

print(f"χ² = {chi2:.3f}")
print(f"p-value = {p:.5f}")
print(f"Ступені свободи = {dof}")
print("Очікувані частоти:\n", expected)

alpha = 0.05
if p < alpha:
    print(f"Висновок: p-value {p:.5f} < α {alpha}, відхиляємо H₀. Є залежність між версією гри та утриманням на 7ий день.")
else:
    print(f"Висновок: p-value {p:.5f} ≥ α {alpha}, не відхиляємо H₀. Статистично значущої залежності немає.")


χ² = 9.959
p-value = 0.00160
Ступені свободи = 1
Очікувані частоти:
 [[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]
Висновок: p-value 0.00160 < α 0.05, відхиляємо H₀. Є залежність між версією гри та утриманням на 7ий день.
